In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
from google.colab import files

## Install Transformers library

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 43.2 MB/s 
     |████████████████████████████████| 596 kB 65.5 MB/s 
     |████████████████████████████████| 86 kB 6.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Load the BERT Classifier and Tokenizer along with Input modules

In [4]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from transformers import BertConfig, BertModel

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [5]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


## Download Kaggle dataset

In [6]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/gdrive/MyDrive/Colab\ Notebooks/ml-blockchain/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
! kaggle datasets download -d yasserh/imdb-movie-ratings-sentiment-analysis
! unzip imdb-movie-ratings-sentiment-analysis.zip

 44% 9.00M/20.6M [00:00<00:00, 54.9MB/s]
100% 20.6M/20.6M [00:00<00:00, 98.8MB/s]
Archive:  imdb-movie-ratings-sentiment-analysis.zip
  inflating: movie.csv               


In [8]:
path = '/content/movie.csv'
dataset_file = pd.read_csv(path)

In [9]:
dataset_file.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [10]:

dataset_file.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
dataset_file['DATA_COLUMN'] = dataset_file['DATA_COLUMN'].replace(np.nan, 'Empty')
dataset_file['LABEL_COLUMN'] = dataset_file['LABEL_COLUMN'].replace('positive', 0)
dataset_file['LABEL_COLUMN'] = dataset_file['LABEL_COLUMN'].replace('negative', 1)
dataset_file['LABEL_COLUMN'] = dataset_file['LABEL_COLUMN'].replace('neutral', 2)
dataset_file

,DATA_COLUMN,LABEL_COLUMN
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


## Split train sets

In [11]:
train, test_and_validatition = train_test_split(dataset_file, test_size=0.2, random_state=77)
test, validation = train_test_split(test_and_validatition, test_size=0.5, random_state=77)
print(len(train),len(test),len(validation))

32000 4000 4000


## Save test and validation datasets

In [12]:
model_save_name = 'BERTModel-IMDB'
path = F"/content/gdrive/MyDrive/Colab Notebooks/ml-blockchain/savedModels/{model_save_name}" 
testSetPath = F"/content/gdrive/MyDrive/Colab Notebooks/ml-blockchain/savedModels/{model_save_name}-test.csv"
validationSetPath = F"/content/gdrive/MyDrive/Colab Notebooks/ml-blockchain/savedModels/{model_save_name}-validation.csv"

In [13]:
with open(testSetPath, 'w', encoding = 'utf-8-sig') as f:
  test.to_csv(f)

with open(validationSetPath, 'w', encoding = 'utf-8-sig') as f:
  validation.to_csv(f)

## Create input sequences

In [14]:
def convert_data_to_examples_single(inputDataset, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = inputDataset.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)  
  return train_InputExamples


def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [15]:
DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [16]:
test_inputExamples = convert_data_to_examples_single(test, DATA_COLUMN, LABEL_COLUMN)
test_data = convert_examples_to_tf_dataset(list(test_inputExamples), tokenizer)
test_data = test_data.batch(32)

validation_InputExamples = convert_data_to_examples_single(validation, DATA_COLUMN, LABEL_COLUMN)
validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


## Configure the Loaded BERT model and Train for Fine-tuning

In [17]:
train_InputExamples  = convert_data_to_examples_single(train, DATA_COLUMN, LABEL_COLUMN)
train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [18]:
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=3e-5, epsilon=1e-08), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
2000/2000 [==============================] - 1708s 842ms/step - loss: 0.2586 - accuracy: 0.8898 - val_loss: 0.4870 - val_accuracy: 0.8482
Epoch 2/2
2000/2000 [==============================] - 1683s 841ms/step - loss: 0.0829 - accuracy: 0.9718 - val_loss: 0.5143 - val_accuracy: 0.8920


In [19]:
model.evaluate(test_data)

125/125 [==============================] - 35s 282ms/step - loss: 0.5178 - accuracy: 0.8920


[0.5177902579307556, 0.8920000195503235]

In [20]:
model.save_pretrained(path)

## Load model

In [21]:
loaded_model = TFBertForSequenceClassification.from_pretrained(path, local_files_only=True)
loaded_model.summary()

Some layers from the model checkpoint at /content/gdrive/MyDrive/Colab Notebooks/ml-blockchain/savedModels/BERTModel-IMDB were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/gdrive/MyDrive/Colab Notebooks/ml-blockchain/savedModels/BERTModel-IMDB.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassi

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


## Make Predictions with the Fine-tuned Model

In [26]:
pred_sentences = ['Terrible movie',
                  'I wasted 2 hours of my life',
                  'I will watch it a million times',
                  'I Absolutely loved it']

In [27]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = loaded_model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

Terrible movie : 
 Negative
I wasted 2 hours of my life : 
 Negative
I will watch it a million times : 
 Positive
I Absolutely loved it : 
 Positive
